 # Pokemon Image Clustering #

In [1]:
# for loading/processing the images
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input
# for other things
import random
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
import os

pokemons = []
rootdir = '../data/PokemonData'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        pokemons.append([file, subdir.split('\\')[-1]])
        
print(pokemons[:10])

[['0282b2f3a22745f1a436054ea15a0ae5.jpg', 'Abra'], ['06b9eec4827d4d49b1b4c284308708df.jpg', 'Abra'], ['10a9f06ec6524c66b779ea80354f8519.jpg', 'Abra'], ['1788abb8b51f48509cfac8067bd99e14.jpg', 'Abra'], ['28cfad92ad934d1f9b579cbff4b5d012.jpg', 'Abra'], ['2eb2a528f9a247358452b3c740df69a0.jpg', 'Abra'], ['2fd28e699b7c4208acd1637fbad5df2d.jpeg', 'Abra'], ['32240b108a8140f8b31c495166fc453c.jpg', 'Abra'], ['34532bb006714727ade4075f0a72b92d.jpg', 'Abra'], ['3680c3f65a484c3ba05a7cb93e1d7ae3.jpg', 'Abra']]


In [3]:
pokemon_labels = pd.DataFrame(pokemons, columns = ['FileName', 'Label'])
# credit to https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34
#    for help with image processing
# load the image as a 224x224 array
import matplotlib.image as mpimg
import imghdr
img_dict = {}
for pokemon in range(len(pokemon_labels)):
    img_path = '../data/PokemonData/'+pokemon_labels.iloc[pokemon]['Label']+'/'+pokemon_labels.iloc[pokemon]['FileName']
    img_type = imghdr.what(img_path)
    if img_type != "png" and img_type != "jpg" and img_type != "jpeg":
        os.remove(img_path)
    else:
        img = load_img(img_path, target_size=(224,224))
        # convert from 'PIL.Image.Image' to numpy array
        img = np.array(img)
        reshaped_img = img.reshape(224,224,3)
        x = preprocess_input(reshaped_img)
        if pokemon_labels.iloc[pokemon]['Label'] in img_dict.keys(): img_dict[pokemon_labels.iloc[pokemon]['Label']].append(x)
        else : img_dict[pokemon_labels.iloc[pokemon]['Label']] = [x]

In [ ]:
class img_K_means:
    def __init__(self,img_dict,K,dist_func):
        self.imgs       = img_dict #dictionary
        self.clustroids = np.asarray(random.sample(img_dict.keys(), K))
        self.centroids  = [224,224,3]*K
        self.distances  = np.zeros([K])
        self.clusters   = {}
        self.k = K
        self.dist_func  = dist_func
        for i in self.clustroids: self.clusters[i] = [i]
        for i in range(K): self.centroids[i] = random.choice(img_dict[self.clustroids[i]])
        print(zip(self.imgs.keys(),self.imgs.values()))
        
    def cluster(self):
        for img_label,img_val in zip(self.imgs.keys(),self.imgs.values()):        # For each data point
            for j, clustroid in enumerate(self.clustroids):                       # go through each clustroid
                self.distances[j] = self.squared_dist(img_val,self.centroids[j])  # and calculate distance to clustroid.
            index = np.argmin(self.distances)                                     # Get the index of the closest clustroid
            if self.clustroids[index] in self.clusters.keys(): self.clusters[self.clustroids[index]].append([img_label,img_val])
            else: self.clusters[self.clustroids[index]] = [[img_label,img_val]]   #assign to corresponding cluster
            print(self.clusters[self.clustroids[index]])
                
    def compute_centroids(self):
        distances = np.zeros([1,len(self.imgs.keys())])
        self.clusters = {}
        for index,cluster in enumerate(self.clusters.keys()):
            centroid = np.zeros([224,224,3])
            for img_val in self.clusters[cluster][1]: centroid += img_val
            self.centroids[index] = centroid/len(self.clusters[cluster])
            for i, img_val in enumerate(self.imgs.values()): distances[i] = self.squared_dist(img_val,clustroid)
            index = np.argmin(distances)
            if img_labels[index] in self.clusters.keys(): print("FUUUUUUUUUUCK")
            else: self.clusters[img_label[index]] = img_label[index]
        
    def squared_dist(self,img1,img2):
        return np.sqrt(np.sum((img1-img2)**2))
    
    def fit(self):
        cloysters = self.clusters
        for i in range(1000):
            self.cluster()
            self.compute_centroids()
            if cloysters == self.clusters: break
            cloysters = self.clusters
            
first_try = img_K_means(img_dict,150,0)
first_try.fit()
                

            

['Weepinbell', ['Abra', [array([[[ 36.060997 ,   2.2210007, 127.32     ],
        [ 36.060997 ,   2.2210007, 127.32     ],
        [ 39.060997 ,   5.2210007, 125.32     ],
        ...,
        [ 98.061    ,  16.221    , 124.32     ],
        [ 98.061    ,  16.221    , 124.32     ],
        [ 97.061    ,  15.221001 , 123.32     ]],

       [[ 34.060997 ,   3.2210007, 127.32     ],
        [ 35.060997 ,   4.2210007, 126.32     ],
        [ 37.060997 ,   5.2210007, 125.32     ],
        ...,
        [103.061    ,  21.221    , 129.32     ],
        [103.061    ,  21.221    , 129.32     ],
        [102.061    ,  20.221    , 128.32     ]],

       [[ 33.060997 ,   4.2210007, 128.32     ],
        [ 33.060997 ,   5.2210007, 126.32     ],
        [ 36.060997 ,   6.2210007, 126.32     ],
        ...,
        [100.061    ,  22.221    , 131.32     ],
        [ 99.061    ,  21.221    , 130.32     ],
        [ 99.061    ,  21.221    , 130.32     ]],

       ...,

       [[ 93.061    ,  50.221    , 

['MrMime', ['Aerodactyl', [array([[[  12.060997  ,    1.2210007 ,    4.3199997 ],
        [ -31.939003  ,  -42.779     ,  -39.68      ],
        [  32.060997  ,   23.221     ,   27.32      ],
        ...,
        [ -74.939     ,  -91.779     ,  -26.68      ],
        [ -92.939     , -108.779     ,  -36.68      ],
        [ -96.939     , -109.779     ,  -38.68      ]],

       [[ -99.939     , -111.779     , -108.68      ],
        [-103.939     , -116.779     , -113.68      ],
        [-103.939     , -116.779     , -115.68      ],
        ...,
        [ -59.939003  ,  -82.779     ,  -19.68      ],
        [-103.939     , -116.779     ,  -57.68      ],
        [ -73.939     ,  -90.779     ,  -25.68      ]],

       [[  14.060997  ,    0.22100067,    2.3199997 ],
        [ 148.061     ,  134.22101   ,  131.32      ],
        [  90.061     ,   78.221     ,   81.32      ],
        ...,
        [ -19.939003  ,  -45.779     ,    9.32      ],
        [ -41.939003  ,  -66.779     ,   -7.680000

['Weepinbell', ['Abra', [array([[[ 36.060997 ,   2.2210007, 127.32     ],
        [ 36.060997 ,   2.2210007, 127.32     ],
        [ 39.060997 ,   5.2210007, 125.32     ],
        ...,
        [ 98.061    ,  16.221    , 124.32     ],
        [ 98.061    ,  16.221    , 124.32     ],
        [ 97.061    ,  15.221001 , 123.32     ]],

       [[ 34.060997 ,   3.2210007, 127.32     ],
        [ 35.060997 ,   4.2210007, 126.32     ],
        [ 37.060997 ,   5.2210007, 125.32     ],
        ...,
        [103.061    ,  21.221    , 129.32     ],
        [103.061    ,  21.221    , 129.32     ],
        [102.061    ,  20.221    , 128.32     ]],

       [[ 33.060997 ,   4.2210007, 128.32     ],
        [ 33.060997 ,   5.2210007, 126.32     ],
        [ 36.060997 ,   6.2210007, 126.32     ],
        ...,
        [100.061    ,  22.221    , 131.32     ],
        [ 99.061    ,  21.221    , 130.32     ],
        [ 99.061    ,  21.221    , 130.32     ]],

       ...,

       [[ 93.061    ,  50.221    , 

['MrMime', ['Aerodactyl', [array([[[  12.060997  ,    1.2210007 ,    4.3199997 ],
        [ -31.939003  ,  -42.779     ,  -39.68      ],
        [  32.060997  ,   23.221     ,   27.32      ],
        ...,
        [ -74.939     ,  -91.779     ,  -26.68      ],
        [ -92.939     , -108.779     ,  -36.68      ],
        [ -96.939     , -109.779     ,  -38.68      ]],

       [[ -99.939     , -111.779     , -108.68      ],
        [-103.939     , -116.779     , -113.68      ],
        [-103.939     , -116.779     , -115.68      ],
        ...,
        [ -59.939003  ,  -82.779     ,  -19.68      ],
        [-103.939     , -116.779     ,  -57.68      ],
        [ -73.939     ,  -90.779     ,  -25.68      ]],

       [[  14.060997  ,    0.22100067,    2.3199997 ],
        [ 148.061     ,  134.22101   ,  131.32      ],
        [  90.061     ,   78.221     ,   81.32      ],
        ...,
        [ -19.939003  ,  -45.779     ,    9.32      ],
        [ -41.939003  ,  -66.779     ,   -7.680000

['Haunter', ['Arbok', [array([[[-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        ...,
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ]],

       [[-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        ...,
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ]],

       [[-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        ...,
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ]],

       ...,

       [[-103.939, -116.779, -123.68 ],
        [ -97.939, -112.779, -119.68 ],
        [ -96.939, -116.779, -123.68 ],
        ...,
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.77

['Vulpix', ['Arcanine', [array([[[-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ],
        ...,
        [-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ]],

       [[-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ],
        ...,
        [-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ]],

       [[-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ],
        ...,
        [-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ],
        [-103.939   , -116.779   , -123.68    ]],

       ...,

       [[ -94.939   , -113.779   , 

['MrMime', ['Aerodactyl', [array([[[  12.060997  ,    1.2210007 ,    4.3199997 ],
        [ -31.939003  ,  -42.779     ,  -39.68      ],
        [  32.060997  ,   23.221     ,   27.32      ],
        ...,
        [ -74.939     ,  -91.779     ,  -26.68      ],
        [ -92.939     , -108.779     ,  -36.68      ],
        [ -96.939     , -109.779     ,  -38.68      ]],

       [[ -99.939     , -111.779     , -108.68      ],
        [-103.939     , -116.779     , -113.68      ],
        [-103.939     , -116.779     , -115.68      ],
        ...,
        [ -59.939003  ,  -82.779     ,  -19.68      ],
        [-103.939     , -116.779     ,  -57.68      ],
        [ -73.939     ,  -90.779     ,  -25.68      ]],

       [[  14.060997  ,    0.22100067,    2.3199997 ],
        [ 148.061     ,  134.22101   ,  131.32      ],
        [  90.061     ,   78.221     ,   81.32      ],
        ...,
        [ -19.939003  ,  -45.779     ,    9.32      ],
        [ -41.939003  ,  -66.779     ,   -7.680000

['Haunter', ['Arbok', [array([[[-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        ...,
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ]],

       [[-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        ...,
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ]],

       [[-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        ...,
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ]],

       ...,

       [[-103.939, -116.779, -123.68 ],
        [ -97.939, -112.779, -119.68 ],
        [ -96.939, -116.779, -123.68 ],
        ...,
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.77

['Squirtle', ['Bellsprout', [array([[[ -99.939, -116.779, -123.68 ],
        [ -99.939, -116.779, -123.68 ],
        [ -97.939, -115.779, -121.68 ],
        ...,
        [ -99.939, -114.779, -121.68 ],
        [-100.939, -115.779, -122.68 ],
        [ -99.939, -114.779, -121.68 ]],

       [[ -94.939, -111.779, -119.68 ],
        [ -96.939, -113.779, -121.68 ],
        [ -99.939, -116.779, -123.68 ],
        ...,
        [-101.939, -116.779, -123.68 ],
        [-101.939, -116.779, -123.68 ],
        [-101.939, -116.779, -122.68 ]],

       [[-101.939, -116.779, -123.68 ],
        [-100.939, -115.779, -122.68 ],
        [ -97.939, -112.779, -119.68 ],
        ...,
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -122.68 ]],

       ...,

       [[-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        [-103.939, -116.779, -123.68 ],
        ...,
        [-103.939, -116.779, -123.68 ],
        [-103.939, -

['MrMime', ['Aerodactyl', [array([[[  12.060997  ,    1.2210007 ,    4.3199997 ],
        [ -31.939003  ,  -42.779     ,  -39.68      ],
        [  32.060997  ,   23.221     ,   27.32      ],
        ...,
        [ -74.939     ,  -91.779     ,  -26.68      ],
        [ -92.939     , -108.779     ,  -36.68      ],
        [ -96.939     , -109.779     ,  -38.68      ]],

       [[ -99.939     , -111.779     , -108.68      ],
        [-103.939     , -116.779     , -113.68      ],
        [-103.939     , -116.779     , -115.68      ],
        ...,
        [ -59.939003  ,  -82.779     ,  -19.68      ],
        [-103.939     , -116.779     ,  -57.68      ],
        [ -73.939     ,  -90.779     ,  -25.68      ]],

       [[  14.060997  ,    0.22100067,    2.3199997 ],
        [ 148.061     ,  134.22101   ,  131.32      ],
        [  90.061     ,   78.221     ,   81.32      ],
        ...,
        [ -19.939003  ,  -45.779     ,    9.32      ],
        [ -41.939003  ,  -66.779     ,   -7.680000

In [ ]:
print(first_try.clusters)